In [2]:
!pip install onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import onnx
import onnxruntime as ort

In [10]:

# --- 1. Hyperparameters ---
ONNX_MODEL_PATH = "lstm_food_freshness.onnx"
DATA_FILE = "https://raw.githubusercontent.com/PenyelamatPangan/Models/main/food_freshness_dataset.csv"


SEQUENCE_LENGTH = 10  # How many time steps to look back
BATCH_SIZE = 64
NUM_EPOCHS = 30 # MODIFIED: Increased epochs
LEARNING_RATE = 0.001
HIDDEN_SIZE = 32      # MODIFIED: Simplified model
NUM_LAYERS = 1        # MODIFIED: Simplified model

In [5]:

# --- 2. Helper Function to Create Sequences ---
def create_sequences(features, labels, seq_length):
    """
    Creates sliding window sequences from tabular data.
    """
    X_seq, y_seq = [], []
    for i in range(len(features) - seq_length):
        X_seq.append(features[i:(i + seq_length)])
        y_seq.append(labels[i + seq_length - 1]) # Label is from the last item in the window
    return np.array(X_seq), np.array(y_seq)

In [11]:
print("Starting LSTM training and ONNX export...")

# --- 3. Load and Preprocess Data ---
try:
    df = pd.read_csv(DATA_FILE)
except FileNotFoundError:
    print(f"Error: '{DATA_FILE}' not found.")
    print("Please run the data generation script from the previous step first.")
    exit()

print(f"Loaded {len(df)} rows from {DATA_FILE}.")

# Separate features (X) and labels (y)
features = ['C2H5OH_PPM', 'NH3_PPM', 'CH4_PPM', 'MQ_Analog_Value']
target = 'Output'

X = df[features].values
y = df[target].values

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create sequences
# This is the most important step for an LSTM
X_seq, y_seq = create_sequences(X_scaled, y, SEQUENCE_LENGTH)

print(f"Created {len(X_seq)} sequences of length {SEQUENCE_LENGTH}.")

# Split into training and testing sets
X_train_seq, X_test_seq, y_train, y_test = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42
)

# Convert to PyTorch Tensors
# Shape required for LSTM: (batch_size, sequence_length, num_features)
X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


Starting LSTM training and ONNX export...
Loaded 100000 rows from https://raw.githubusercontent.com/PenyelamatPangan/Models/main/food_freshness_dataset.csv.
Created 99990 sequences of length 10.


In [12]:

# --- 4. Define the LSTM Model ---
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=1):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM layer
        # batch_first=True makes input shape (batch_size, seq_length, input_size)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=0.2)

        # Fully connected layer to map LSTM output to our desired output
        self.fc = nn.Linear(hidden_size, output_size)

        # Sigmoid for final probability
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Initialize hidden state and cell state
        # (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        # out shape: (batch_size, seq_length, hidden_size)
        out, _ = self.lstm(x, (h0, c0))

        # We only care about the output of the *last* time step
        out = out[:, -1, :]

        # Pass last output through the fully connected layer
        out = self.fc(out)

        # Apply sigmoid for classification
        # Note: For ONNX export, we include sigmoid in the model.
        # For training, BCEWithLogitsLoss is better (it has sigmoid built-in).
        if not self.training:
            out = self.sigmoid(out)

        return out

# Instantiate model, loss, and optimizer
input_size = len(features)  # 4 features
model = LSTMClassifier(input_size, HIDDEN_SIZE, NUM_LAYERS)
criterion = nn.BCEWithLogitsLoss() # Numerically stable, includes sigmoid
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [13]:

# --- 5. Train the Model ---
print(f"Starting training for {NUM_EPOCHS} epochs...")
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    for i, (seqs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(seqs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {total_loss/len(train_loader):.4f}')

print("Training complete.")


Starting training for 30 epochs...
Epoch [1/30], Loss: 0.0963
Epoch [2/30], Loss: 0.0139
Epoch [3/30], Loss: 0.0089
Epoch [4/30], Loss: 0.0050
Epoch [5/30], Loss: 0.0023
Epoch [6/30], Loss: 0.0010
Epoch [7/30], Loss: 0.0005
Epoch [8/30], Loss: 0.0002
Epoch [9/30], Loss: 0.0001
Epoch [10/30], Loss: 0.0001
Epoch [11/30], Loss: 0.0000
Epoch [12/30], Loss: 0.0000
Epoch [13/30], Loss: 0.0000
Epoch [14/30], Loss: 0.0000
Epoch [15/30], Loss: 0.0000
Epoch [16/30], Loss: 0.0000
Epoch [17/30], Loss: 0.0000
Epoch [18/30], Loss: 0.0000
Epoch [19/30], Loss: 0.0000
Epoch [20/30], Loss: 0.0000
Epoch [21/30], Loss: 0.0000
Epoch [22/30], Loss: 0.0000
Epoch [23/30], Loss: 0.0000
Epoch [24/30], Loss: 0.0000
Epoch [25/30], Loss: 0.0000
Epoch [26/30], Loss: 0.0000
Epoch [27/30], Loss: 0.0000
Epoch [28/30], Loss: 0.0000
Epoch [29/30], Loss: 0.0000
Epoch [30/30], Loss: 0.0000
Training complete.


In [14]:

# --- 6. Evaluate the Model ---
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for seqs, labels in test_loader:
        # Get raw logit outputs
        outputs = model(seqs)

        # Apply sigmoid and threshold to get 0 or 1
        preds = (torch.sigmoid(outputs) > 0.5).float()

        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=['Bad (0)', 'Fresh (1)']))



Test Accuracy: 99.86%

Classification Report:
              precision    recall  f1-score   support

     Bad (0)       1.00      1.00      1.00      9964
   Fresh (1)       1.00      1.00      1.00     10034

    accuracy                           1.00     19998
   macro avg       1.00      1.00      1.00     19998
weighted avg       1.00      1.00      1.00     19998



In [15]:

# --- 7. Export to ONNX ---
print(f"\nExporting model to {ONNX_MODEL_PATH}...")

# Set model to evaluation mode (important for ONNX)
model.eval()

# Create a dummy input matching the model's input shape
# (batch_size, sequence_length, num_features)
# We use a batch size of 1 for the dummy input
dummy_input = torch.randn(1, SEQUENCE_LENGTH, input_size, requires_grad=True)

# Export the model
torch.onnx.export(
    model,                          # The model to export
    dummy_input,                    # A dummy input
    ONNX_MODEL_PATH,                # Where to save the model
    export_params=True,             # Store the trained weights
    opset_version=11,               # ONNX version
    do_constant_folding=True,       # Optimization
    input_names=['input_sequence'], # Name for the input
    output_names=['output'],        # Name for the output
    dynamic_axes={                  # --- VERY IMPORTANT ---
        'input_sequence': {0: 'batch_size'}, # Allows variable batch size
        'output': {0: 'batch_size'}
    }
)

print(f"Successfully exported model to {ONNX_MODEL_PATH}")

# (Optional) Verify the ONNX model
print("Verifying ONNX model...")
onnx_model = onnx.load(ONNX_MODEL_PATH)
onnx.checker.check_model(onnx_model)

# Test with ONNX Runtime
ort_session = ort.InferenceSession(ONNX_MODEL_PATH)
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.detach().numpy()}
ort_outs = ort_session.run(None, ort_inputs)

print("ONNX model verification successful. Inference test output shape:", ort_outs[0].shape)


Exporting model to lstm_food_freshness.onnx...
Successfully exported model to lstm_food_freshness.onnx
Verifying ONNX model...
ONNX model verification successful. Inference test output shape: (1, 1)


/tmp/ipython-input-1477254319.py:13: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
/usr/local/lib/python3.12/dist-packages/torch/onnx/symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the m